### Trabalho 1 - Criptografia
###### Grupo 19

Tiago Passos Rodrigues - A96414

### Enunciado

2. Na criptografia pós-quântica os reticulados inteiros (“hard lattices”) e os problemas a eles associados são uma componente essencial. Um reticulado inteiro pode ser definido por uma matriz $\;\mathsf{L} \in \mathbb{Z}^{m\times n}\;$ (com $\;m > n\;$) de inteiros e por um inteiro primo $\;q\geq 3\;$.
    O chamado problema do vetor curto  (SVP) consiste  no cálculo de um vetor de inteiros
    
$$e \in \{-1,0,1\}^m $$

    não nulo que  verifique a seguinte relação matricial
    
$$\forall\,i < n\,\centerdot\ \sum_{j< m}\,e_j\,\times\,\mathsf{L}_{j,i}\;\equiv\;0\mod q$$


a. Pretende-se resolver o SVP por programação inteira dentro das seguintes condições
    i. Os valores  $m\,n\,q\,$  são escolhidos com $\,n > 30\,$, $\,|m| > 1 + |n|\;$ e $\,|q| > |m|\,$. 
    ii. Os elementos $\;\mathsf{L}_{j,i}\;$ são gerados aleatória e uniformemente no intervalo inteiro $\,\{-d \cdots d\}$ sendo  $\;d\equiv (q-1)/2\;$.
    
b. Pretende-se determinar em, em primeiro lugar, se existe um vetor $\,e\,$ não nulo (pelo menos um dos $\,e_j\,$é diferente de zero). Se existir $\,e\,$ pretende-se calcular o vetor que minimiza o número de componentes não nulas.

### Análise do problema

Temos uma matriz $L_{m,n}$ com tamanho $M\times N$ de números inteiros. Queremos calcular o vetor $e$ que:
 
$$\forall_{i < N} \sum_{j< M} e_{j} * L_{j,i} = Q * k $$

### Implementação

Começamos por importar a biblioteca de uma ferramenta de programação inteira do OR-Tools que consegue lidar com a multiplicação de variáveis e criar uma instância do *model*.


Depois inicializamos o *model* `criptografia` e definir os valores para as constantes $M$, $N$ e $Q$ com as restrições referidas em cima.

In [1]:
# Importar biblioteca
from ortools.sat.python import cp_model

# Cria o modelo CP-SAT
model = cp_model.CpModel()

### Criação das variáveis e Inputs

In [2]:
# |m| > |n| + 1, |q| > |m|, d = q-1/2

M, N, Q, D = 12, 8, 5, 2  # -> exemplo 1

# M, N, Q, D = 64, 31, 131, 65



# Cria a variável do vetor e
#e = model.NewIntVar(-1, 1, 'e')
e = {}
for m in range(M):
    e[m] = model.NewIntVar(-1, 1, 'e')


x = {} #variavel auxiliar
for m in range(M):
    x[m] = model.NewIntVar(0, 1, 'x') # assume os valores absolutos do e

for m in range(M):
    model.AddAbsEquality(x[m],e[m])
    
model.Add(sum(x[i] for i in range(M)) != 0) # vetor não pode ser nulo

model.Minimize(sum(x[i] for i in range(M))) # minizar os zeros

# importar a libraria numpy para a geração da matriz
import numpy as np
# 1º argumento --> numeros de -D até D, 
# 2º argumento, linha = M, coluna = N
matriz = np.random.randint(-D,D, (M, N))
print(matriz)
#print(matriz[0][1])

k = {} # existe um k
for i in range(N):
    k[i] = model.NewIntVar(-100, 100, 'k')

for i in range(N):
    model.Add(sum(e[j] * matriz[j][i] for j in range(M)) == Q*k[i]) #percorre a coluna


[[ 1 -2 -2 -1 -2  0 -1 -1]
 [-1 -2 -2 -2  0  1  0 -1]
 [ 1 -2  0  1 -1 -1 -2  0]
 [-2  1 -1  1 -1 -1  1  1]
 [-1 -2 -1 -1 -2 -1  1 -2]
 [ 1  1  0 -1 -2  0  1  1]
 [-2  0 -2  1 -2 -2 -1  1]
 [ 0 -2  0 -2  1 -2  0  1]
 [ 1  1  0 -1 -1  1  0 -2]
 [-1 -1 -2  1 -1  1  1 -1]
 [ 1  0 -2  1 -1 -2 -2 -2]
 [-1 -2 -1 -1 -2 -2  1 -1]]


### Criação do Solver e interpretação dos resultados

In [3]:
# Cria um solver CP-SAT a solver and solves the model.
solver = cp_model.CpSolver()

# Invoca o solver com o modelo criado
status = solver.Solve(model)

# Interpreta os resultados
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    #print('x = %i' % solver.Value(x))
    print("Valor do vetor e:")
    for m in range(M):
        print(solver.Value(e[m]), end=' ')
else:
    print('No solution found.')


Valor do vetor e:
1 1 0 0 0 0 0 0 1 0 0 1 